In [1]:
import os
from dotenv import load_dotenv

In [2]:
raw = os.getenv("GROQ_API_KEY")

In [3]:
from utils.config_loader import load_config

In [4]:
config = load_config()

In [5]:
config.get("providers", {}).get("embedding")

'openai'

In [10]:
with open("./data/Rudy-2025.pdf", "rb") as f:
    content = f.read()
print(content)

b'%PDF-1.3\n%\xc7\xec\x8f\xa2\n5 0 obj\n<</Length 6 0 R/Filter /FlateDecode>>\nstream\nx\x9c\xd5XIs\x1d5\x10\xbe\xbf_17f(FVk\x17\'\x88m\xb2\x00&8.R\x90p\xb0\xfd\xbcU\xc5Kb\x1b\xf0\xbf\xa75\xf3F\xdd\xe3\xd7I|0\x14\xd8\x97.\x8d\xd4\xdb\xf7u\xab\xf5\xde7Z\x81it\xf9\x9f\x84\xc3\xf3\xc5\xc6nlN\xae\x17\xc3r\xb3\xfbt%|8Y\xbc_$e\xcb\xdf\xb0\xc0\xe5\xc3\xf3\xe6\xc9\x1e\x1e\xcc\rx\x95C\xb3w\xbc\x80\xe1\x034F\x83J\xae\x89\xda\xa9\x88\x1f\xce\x17o\xdao;\xab\x95\x89:\xb6\xd7]\xaf\x95\xcd:g3\x93/;P9\x86\x9c\xdbC\xb6zVd\xc8\xc1\x81k\xf7\xd9\xfaMg\x14D\xeb\xda\xa3\x8f\xa8k\xd8\xc1\xd1x\xc8\xa9\xbd\xa8F\xb8\xe2s\\\x8d\xc98\x98\x99x\xc7vpm\xcf\xba":\x1f\x13\xf3\x99\x9b\xbe\x12\x8dT\x8f\xe76LL\xbf\xef\xbd\x18\xf3\xe8\x94v%\x8f\xce)\xd7\xf4\x10\x94\xc7\xec-1{\x06M:\xfc\x0b\xd0\x02\x89\xe8\x14\x9e\xf0\xc6\x9a\xf6\x15\xae"0)\xa4\xf6\xb6s\n\x82\xc9\x80a\xa1q\xd0>;.\x1e\x91\x82\x0fE\xc1\xb8W\xd6u3\xb8:\xcal3S\xc1\xc4\x9b\x0ew\x96Xzc\x957C\x00:\x8d\x01\xbc\xee\xacJ&\xeb\x99\xf9k~\x96\x0c\x1d\xd4\x00.\xab\xc4\x

In [12]:
import fitz  # PyMuPDF

In [18]:
text_chunks = []
with fitz.open("./data/Rudy-2025.pdf") as doc:
    print(doc.page_count)
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text_chunks.append(f"\n--- Page {page_num + 1} ---\n{page.get_text()}")  # type: ignore
text = "\n".join(text_chunks)

1


ValueError: document closed

In [15]:
print(text)


--- Page 1 ---
Associates Animal Hospital
21 Summer Street
Westborough, MA  01581
(508) 870-0600
associatesah@yourvetdoc.com
VACCINE CERTIFICATE
Owner's Name:
Arindam Choudhury
Date:
2/22/2025
Address:
23 Robert Circle
City:
North Grafton
State:
MA
Zip Code:
01526
Phone Number:
(508) 414-6877
Animal's Name:
Rudy
Species:
Canine
Breed:
Golden Doodle
Color:
Cream
Age:
3 Yrs. 5 Mos.
Sex:
Neutered Male
Weight:
85
VACCINE                    Date Performed:                      
  Due Date:
Rabies K9 3 year
3/3/2023 
3/3/2026
DHPP 3 year
3/3/2023
            
3/3/2026 
Bordetella
2/22/2025
2/22/2026  
 
 
 

